In [ ]:
pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade

In [57]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(role)
print(sess)

arn:aws:iam::756963467680:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [58]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://image-captioning-model/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',                                    # Python version used
)

In [59]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge",
)

-----!

In [60]:
# example request: you always need to define "inputs"
data = {
   "inputs": ['https://huggingface.co/datasets/Narsil/image_dummy/raw/main/parrots.png']
}

# request
predictor.predict(data)[0]

'["a pair of parrots standing next to each other"]'

In [61]:
predictor.endpoint_name

'huggingface-pytorch-inference-2023-12-15-21-16-41-656'

In [62]:
import base64
import os
s3 = boto3.client('s3')
response = s3.list_objects_v2(Bucket='memerr-memes')

# Print the list of objects
if 'Contents' in response:
    for obj in response['Contents']:
        object_key = obj['Key']

        if 'jpg' or 'png' in object_key.lower():
            img_response = s3.get_object(Bucket='memerr-memes', Key=object_key)
            file_path = os.path.join('./', 'image.png')
            with open(file_path, 'wb') as f:
                s3.download_fileobj('memerr-memes', object_key, f)

            with open("./image.png", "rb") as f:
                image_data = f.read()
            # image_data = img_response['Body'].read()
            base64_image = base64.b64encode(image_data).decode('utf-8')

            print(object_key, type(base64_image))
            break
            # result = process_image(base64_image)

000df100-cb97-4a12-b5ec-4dbc48c0dfff_image_3704.jpg <class 'str'>


In [84]:
import json
input_data = {"inputs": [base64_image]}
runtime= boto3.client('runtime.sagemaker')
response = runtime.invoke_endpoint(EndpointName=predictor.endpoint_name,
                                       ContentType='application/json',
                                       Body=json.dumps(input_data))
print(response)

{'ResponseMetadata': {'RequestId': '41b455ae-1e3f-4d30-a1b1-78a830e96bc0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '41b455ae-1e3f-4d30-a1b1-78a830e96bc0', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Fri, 15 Dec 2023 21:32:06 GMT', 'content-type': 'application/json', 'content-length': '80', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f351e836c20>}


In [85]:
# Read the content as bytes
content_bytes = response['Body'].read()

# Decode bytes into text using the appropriate encoding (e.g., UTF-8)
content_text = content_bytes.decode('utf-8')
print(content_bytes)
print((response['Body'].read().decode('utf-8')))

b'[\n  "[\\"a man in a suit and tie posing for a picture\\"]",\n  "application/json"\n]'

